## Exercise 04. Enrichment and transformations

1. Считываю файл JSON, который вы сохранили в упражнении ex02.

In [38]:
import pandas as pd
import numpy as np

df = pd.read_json("../data/auto.json")

pd.options.display.float_format = "{:.2f}".format

df.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus


2. Обогащаю фрейм данных, используя выборку из этого фрейма данных.

In [39]:
sample = df.sample(n=200, random_state=21, replace=True)

In [40]:
concat_rows = pd.concat([df, sample], ignore_index=True)

3. Добаляю в фрейм данных concat_rows новый столбец, содержащий сгенерированные данные.

In [41]:
np.random.seed(21)
years = np.random.randint(1980, 2020, size=len(concat_rows))

concat_rows['Year'] = years
concat_rows.head()


,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014


4. Дополняю фрейм данных информацией из другого фрейма данных.

In [42]:
surnames_df = pd.read_json("../../datasets/surname.json")

surnames_df = pd.DataFrame(surnames_df.values[1:], columns=surnames_df.iloc[0])

In [43]:

surnames = (surnames_df["NAME"].str.replace(r"[^A-Za-z]", "", regex=True).str.strip())


unique_cars = concat_rows["CarNumber"].unique()


np.random.seed(21)
owner_surnames = np.random.choice(surnames, size=len(unique_cars), replace=True)


owners = pd.DataFrame({
    "CarNumber": unique_cars,
    "SURNAME": owner_surnames
})

owners.head()


,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ


In [44]:

new_fines = pd.DataFrame({
    "CarNumber": ["12RW12RUS", "34VG34RUS", "56HK56RUS", "78XK78RUS", "90AZ90RUS"],
    "Refund": [1.0, 2.0, 1.0, 2.0, 1.0],
    "Fines": [5000, 6000, 7000, 8000, 9000],
    "Make": ["Tesla", "BMW", "Lada", "Ford", "Toyota"],
    "Model": ["Model_S", "X5", "Largus", "Focus", "Camry"],
    "Year": [2015, 2016, 2017, 2018, 2019]
})

fines = pd.concat([concat_rows, new_fines], ignore_index=True)

owners = owners.iloc[:-20]

new_owners = pd.DataFrame({
    "CarNumber": ["3553621XX33RUS", "19034FWON37RUS", "532JFN312RUS"],
    "SURNAME": ["Smith", "Johnson", "Williams"]
})

owners = pd.concat([owners, new_owners], ignore_index=True)


In [45]:
merge_inner = pd.merge(fines, owners, on="CarNumber", how="inner")

merge_outer = pd.merge(fines, owners, on="CarNumber", how="outer")

merge_right = pd.merge(fines, owners, on="CarNumber", how="right")

merge_left = pd.merge(fines, owners, on="CarNumber", how="left")

5. Создаю сводную таблицу на основе `fines` фрейма данных

In [46]:
pivot = fines.pivot_table(index=["Make","Model"], columns="Year", values="Fines", aggfunc="sum")
pivot.head()

Year            1980      1981      1982     1983      1984      1985  \
Make Model                                                              
Audi             NaN       NaN       NaN      NaN       NaN       NaN   
BMW              NaN       NaN       NaN      NaN       NaN       NaN   
     X5          NaN       NaN       NaN      NaN       NaN       NaN   
Ford Focus  62394.59 395589.17 140383.76 63100.00 111294.59 189583.76   
     Mondeo      NaN       NaN       NaN      NaN       NaN       NaN   

Year            1986      1987     1988      1989  ...      2010     2011  \
Make Model                                         ...                      
Audi             NaN       NaN      NaN       NaN  ...       NaN      NaN   
BMW              NaN       NaN      NaN       NaN  ...       NaN      NaN   
     X5          NaN       NaN      NaN       NaN  ...       NaN      NaN   
Ford Focus  88994.59 121900.00 95989.17 115500.00  ... 120183.76 86689.17   
     Mondeo      NaN       NaN      NaN   8600.00  ...       NaN      NaN   

Year             2012      2013      2014      2015     2016      2017  \
Make Model                                                               
Audi              NaN       NaN       NaN       NaN      NaN       NaN   
BMW           3000.00       NaN       NaN   8594.59      NaN       NaN   
     X5           NaN       NaN       NaN       NaN  6000.00       NaN   
Ford Focus  112700.00 145494.59 158894.59 203694.59 72594.59 272200.00   
     Mondeo  34400.00       NaN       NaN       NaN 46200.00       NaN   

Year             2018      2019  
Make Model                       
Audi              NaN       NaN  
BMW           6500.00       NaN  
     X5           NaN       NaN  
Ford Focus  293194.59 101100.00  
     Mondeo       NaN       NaN  

[5 rows x 40 columns]

6. Сохраняю оба `fines` и `owners` фрейма данных в файлы CSV без индекса.

In [47]:
fines.to_csv("../data/fines.csv", index=False)
owners.to_csv("../data/owners.csv", index=False)
